### MaxFuse running on cortical data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import anndata as ad
import scanpy as sc

import sys
sys.path.append("../../MaxFuse_devo/09302022V/")
import match
import metrics
from scipy.io import mmread
import os

In [2]:
meta_rna = pd.read_csv('/atac_bench_nrz/greanleaf_cortical/data/greenleaf_cortical_meta.csv')
celltype_labels_rna = meta_rna['celltype'].to_numpy()
np.unique(celltype_labels_rna)

array(['Cyc. Prog.', 'EC/Peric.', 'GluN2', 'GluN3', 'GluN4', 'GluN5',
       'IN1', 'IN2', 'IN3', 'RG', 'SP', 'mGPC/OPC', 'nIPC/GluN1'],
      dtype=object)

In [3]:
# read in the counts
rna = mmread("/atac_bench_nrz/greanleaf_cortical/data/greenleaf_cortical_SCT_data.mtx").tocsr()
rna.shape

(8981, 16335)

In [4]:
# read in the names
rna_names = pd.read_csv(
    '/atac_bench_nrz/greanleaf_cortical/data/greenleaf_cortical_rna_names.csv'
)['names'].to_numpy()

In [5]:
# read activity
atacactivity=mmread("/atac_bench_nrz/greanleaf_cortical/data/greenleaf_cortical_GENEACTIVITY_data.mtx").tocsr()
atacactivity.shape

(8981, 16335)

In [6]:
# activity names
gas_names = rna_names

In [7]:
peak_lsi = pd.read_csv('/atac_bench_nrz/greanleaf_cortical/data/greenleaf_cortical_peak_lsi.csv')
peak_lsi = peak_lsi.iloc[:,:49]

In [8]:
## make rna anndata
rna_adata = ad.AnnData(
    rna.tocsr(), dtype=np.float32
)
rna_adata.var_names = rna_names
rna_adata.obs_names = peak_lsi.index.to_list() # get the bardoces
rna_adata

AnnData object with n_obs × n_vars = 8981 × 16335

In [9]:
## make GAS anndata
activity_adata = ad.AnnData(atacactivity, dtype=np.float32)
activity_adata.var_names = gas_names
activity_adata

AnnData object with n_obs × n_vars = 8981 × 16335

In [10]:
peak_adata=ad.AnnData(peak_lsi, dtype=np.float32)
peak_adata

AnnData object with n_obs × n_vars = 8981 × 49

In [11]:
shared_genes = np.intersect1d(rna_adata.var_names, activity_adata.var_names)
len(np.intersect1d(rna_adata.var_names, activity_adata.var_names))

16335

In [12]:
rna_shared = rna_adata[:, shared_genes].X.todense()
activity_shared = activity_adata[:, shared_genes].X.todense()

mask = ((rna_shared.std(axis=0) > 0.01) & (activity_shared.std(axis=0) > 0.0)).A1 # filter out static ones
rna_shared = rna_shared[:, mask]
activity_shared = activity_shared[:, mask]

In [13]:
print((rna_shared.shape, activity_shared.shape))

((8981, 15669), (8981, 15669))


In [14]:
# input sct norm skip norm process
rna_shared = ad.AnnData(rna_shared)
#sc.pp.normalize_total(rna_shared) #input already normed
#sc.pp.log1p(rna_shared)
sc.pp.highly_variable_genes(rna_shared, n_top_genes = 3000)
sc.pp.scale(rna_shared)
#rna_shared = rna_shared.X

## atac shared
activity_shared = ad.AnnData(activity_shared)
sc.pp.normalize_total(activity_shared)
sc.pp.log1p(activity_shared)
sc.pp.scale(activity_shared)
#activity_shared = activity_shared.X

In [15]:
vgenes = rna_shared.var.highly_variable

In [16]:
# shared features
rnaC_shared = rna_shared[:,vgenes].X
atac_shared = activity_shared[:,vgenes].X
# all features
rnaC_active = rna_shared[:,vgenes].X
atac_active = peak_adata.X

In [17]:
spm = match.MaxFuse(
        shared_arr1=rnaC_shared,
        shared_arr2=atac_shared,
        active_arr1=rnaC_active,
        active_arr2=atac_active,
        method='centroid_shrinkage',
        labels1=None, # if None, then use scanpy clustering pipeline
        labels2=None
    )

In [18]:
spm.split_into_batches(
        max_outward_size=5000,
        matching_ratio=5,
        metacell_size=2,
        method='binning',
        verbose=True,
        seed=42
    )

The first data is split into 2 batches, average batch size is 4490, and max batch size is 4491.
The second data is split into 1 batches, average batch size is 8981, and max batch size is 8981.
Batch to batch correspondence is:
  ['0<->0', '1<->0'].


In [19]:
spm.construct_graphs(
    n_neighbors1=15,
    n_neighbors2=15,
    svd_components1=30,
    svd_components2=15,
    resolution1=2,
    resolution2=2,
    randomized_svd=False,  # @Shuxiao: Had to change this from True to False.  Doesn't work when true, why?
    svd_runs=1,
    resolution_tol=0.1,
    leiden_runs=1,
    leiden_seed=None,
    verbose=True
)

Aggregating cells in arr1 into metacells of average size 2...
Constructing neighborhood graphs for cells in arr1...
Now at batch 0...
Now at batch 1...
Graph construction finished!
Clustering into metacells...
Now at batch 0...
Metacell clustering finished!
Now at batch 1...
Metacell clustering finished!
Constructing neighborhood graphs for cells in arr1...
Now at batch 0...
Now at batch 1...
Graph construction finished!
Clustering the graphs for cells in arr1...
Now at batch 0...
Now at batch 1...
Graph clustering finished!
Constructing neighborhood graphs for cells in arr2...
Now at batch 0...
Graph construction finished!
Clustering the graphs for cells in arr2...
Now at batch 0...
Graph clustering finished!


In [20]:
spm.find_initial_pivots(
    wt1=0.7, wt2=0.7,
    svd_components1=20, svd_components2=20,
    randomized_svd=False, svd_runs=1,
    verbose=True
)

Now at batch 0<->0...
Now at batch 1<->0...
Done!


In [21]:
# test accuracy of initial matching
import utils, metrics
matching = spm._init_matching[0]
labels1 = utils.summarize_clustering(spm._metacell_labels1[0], celltype_labels_rna[spm._batch_to_indices1[0]])
# labels1 = celltype_labels_rna[spm._batch_to_indices1[0]]
labels2 = celltype_labels_rna[spm._batch_to_indices2[0]]
metrics.get_matching_acc(matching, labels1, labels2)

0.6598305840392331

In [22]:
spm.refine_pivots(
    wt1=0.7, wt2=0.7,
    svd_components1=200, svd_components2=None,
    cca_components=18,
    filter_prop=0.,
    n_iters=8,
    randomized_svd=False, 
    svd_runs=1,
    verbose=True
)

Now at batch 0<->0...
Now at batch 1<->0...
Done!


In [23]:
# test accuracy of refined matching
matching = spm._refined_matching[0]
labels1 = utils.summarize_clustering(spm._metacell_labels1[0], celltype_labels_rna[spm._batch_to_indices1[0]])
# labels1 = celltype_labels_rna[spm._batch_to_indices1[0]]
labels2 = celltype_labels_rna[spm._batch_to_indices2[0]]
metrics.get_matching_acc(matching, labels1, labels2)

0.7186803388319215

In [24]:
spm.filter_bad_matches(target='pivot', filter_prop=0.4, verbose=True)

Begin filtering...
Now at batch 0<->0...
Now at batch 1<->0...
2696/4493 pairs of matched cells remain after the filtering.
Fitting CCA on pivots...
Scoring matched pairs...
5447/8981 cells in arr1 are selected as pivots.
2340/8981 cells in arr2 are selected as pivots.
Done!


In [34]:
spm.propagate(
    wt1=0.7,
    wt2=0.7,
    svd_components1=30, 
    svd_components2=15, 
    randomized_svd=False, 
    svd_runs=1, 
    verbose=True
)

Now at batch 0<->0...
Now at batch 1<->0...
Done!


In [35]:
spm.filter_bad_matches(
        target='propagated',
        filter_prop=0.,
        verbose=True
    )

Begin filtering...
Now at batch 0<->0...
Now at batch 1<->0...
17063/17063 pairs of matched cells remain after the filtering.
Scoring matched pairs...
Done!


In [36]:
matching = spm.get_matching(order=(2, 1), target='full_data')
metrics.get_matching_acc(matching, celltype_labels_rna, celltype_labels_rna, order=(2, 1))
# prev 69.5%

0.6948001336154103

In [37]:
full = pd.DataFrame(list(zip(matching[0],matching[1],matching[2])), columns = ["idx1","idx2","score"])
full.to_csv("/atac_bench_nrz/greanleaf_cortical/mf/full_idx_21.csv", index=False)

In [38]:
arr1_cca, arr2_cca = spm.get_embedding(
        active_arr1 = spm.active_arr1,
        active_arr2 = spm.active_arr2,
        refit=False,
        matching=None,
        order=None,
        cca_components=20,
        cca_max_iter=None
    )

In [39]:
out_idx = 20 # save 20 used 15 in downstream analysis
arr1_df = pd.DataFrame(arr1_cca).iloc[:,0:out_idx]
arr2_df = pd.DataFrame(arr2_cca).iloc[:,0:out_idx]
arr1_df.to_csv("/atac_bench_nrz/greanleaf_cortical/mf/full_embed_x0.csv",index=False)
arr2_df.to_csv("/atac_bench_nrz/greanleaf_cortical/mf/full_embed_y0.csv", index=False)